In [2]:
import numpy as np
import torch
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
import matplotlib

from matplotlib import pyplot as plt
import cv2
import supervision as sv
import os

from fpdf import FPDF



In [3]:

CHECKPOINT_PATH = "sam_vit_h_4b8939.pth"

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Device: {DEVICE}")
MODEL_TYPE = "vit_h"

sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH)
sam.to(device=DEVICE)
mask_generator = SamAutomaticMaskGenerator(sam)

Device: cuda:0


In [125]:
# Give the path of your image
image_path = './3c241c3a-c113-419d-9590-4ad1dbf03c74.jpg'
# Read the image from the path
image = cv2.imread(image_path)
# Convert to RGB format
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# Generate segmentation mask
output_mask = mask_generator.generate(image)


In [1]:
def show_output(result_dict,axes=None):
     if axes:
        ax = axes
     else:
        ax = plt.gca()
        ax.set_autoscale_on(False)
     sorted_result = sorted(result_dict, key=(lambda x: x['area']),      reverse=True)
     # Plot for each segment area
     for val in sorted_result:
        mask = val['segmentation']
        img = np.ones((mask.shape[0], mask.shape[1], 3))
        color_mask = np.random.random((1, 3)).tolist()[0]
        for i in range(3):
            img[:,:,i] = color_mask[i]
            ax.imshow(np.dstack((img, mask*0.5)))





In [5]:

def show_output(result_dict,axes=None):
     if axes:
        ax = axes
     else:
        ax = plt.gca()
        ax.set_autoscale_on(False)
     sorted_result = sorted(result_dict, key=(lambda x: x['area']),      reverse=True)
     # Plot for each segment area
     for val in sorted_result:
        mask = val['segmentation']
        img = np.ones((mask.shape[0], mask.shape[1], 3))
        color_mask = np.random.random((1, 3)).tolist()[0]
        for i in range(3):
            img[:,:,i] = color_mask[i]
            ax.imshow(np.dstack((img, mask*0.5)))

def get_second_largest_area(result_dict):
    sorted_result = sorted(result_dict, key=(lambda x: x['area']),      reverse=True)
    return sorted_result[1]



def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)


def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)   


def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2))    


In [8]:
mask_folder = './pad/train/mask/melanoma/'
real_images_folder = './pad/train/data/melanoma/'
result_folder = './result_sam/' 
#get the first image from mask folder, then get the corresponding image from real_images_folder, then generate the mask and save the image with the mask on at result_folder
image_name = os.listdir(real_images_folder)[4]

# Read the image from the path
image = cv2.imread(real_images_folder + image_name)
#if image size is greater than 1920 1080, resize it
if image.shape[0] > 1920 or image.shape[1] > 1080:
    image = cv2.resize(image, (1920, 1080))
    
# Convert to RGB format
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Generate segmentation mask
output_mask = mask_generator.generate(image)

#get second largest area
second_largest_area = get_second_largest_area(output_mask)
print(second_largest_area)
mask = second_largest_area['segmentation']


{'segmentation': array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]]), 'area': 1863, 'bbox': [106, 112, 51, 58], 'predicted_iou': 0.9653923511505127, 'point_coords': [[131.265625, 131.265625]], 'stability_score': 0.953926682472229, 'crop_box': [0, 0, 271, 271]}


In [146]:
mask_folder = './pad/train/mask/melanoma/'
real_images_folder = './pad/train/data/melanoma/'
result_folder = './result_sam/' 

In [147]:
def apply_mask(image, mask, color=None):
    # Convert the mask to a 3 channel image
    if color is None:
        mask_rgb = cv2.cvtColor(mask, cv2.COLOR_GRAY2RGB)
    else:
        mask_rgb = np.zeros_like(image)
        mask_rgb[mask>0] = color

    # Overlay the mask and image
    overlay_image = cv2.addWeighted(image, 0.7, mask_rgb, 0.3, 0)

    return overlay_image

def generate_image_of_mask(output_mask, shape):
    image_rgb = np.zeros((shape[0], shape[1], 3), dtype='uint8')

    for i in range(len(output_mask)):
        mask = output_mask[i]['segmentation']
        mask = np.where(mask, 255, 0).astype('uint8')
        color = np.random.randint(0, 255, 3)
        image = apply_mask(image_rgb, mask, color=color)
        image_rgb = cv2.addWeighted(image_rgb, 1, image, 1, 0)

        

    return image


def process_folder(real_images_folder, result_folder, mask_folder):
    # Initialize PDF
    pdf = FPDF()

    image_names = os.listdir(real_images_folder)
    for image_name in image_names:
        # Read the image from the path
        image = cv2.imread(os.path.join(real_images_folder, image_name))

        # Resize the image if necessary
        if image.shape[0] > 1920 or image.shape[1] > 1080:
            image = cv2.resize(image, (1920, 1080))

        # Convert to RGB format
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Assume mask_generator and get_second_largest_area are defined elsewhere
        # Generate segmentation mask
        output_mask = mask_generator.generate(image)

        overlays_dir = "./overlays/"
        os.makedirs(overlays_dir, exist_ok=True)


        #get the image with the only the mask 
        image_of_masks = generate_image_of_mask(output_mask, image_rgb.shape)
        cv2.imwrite(os.path.join(overlays_dir, image_name), image_of_masks)
        



        # Get second largest area
        second_largest_area = get_second_largest_area(output_mask)
        mask = second_largest_area['segmentation']
        mask = np.where(mask, 255, 0).astype('uint8')

        # Apply the mask to the image
        overlay_image = apply_mask(image_rgb, mask)

        # Create a new page in the PDF
        pdf.add_page()

        # Set the position for the overlay image
        pdf.set_xy(10, 8)
        overlay_image = cv2.cvtColor(overlay_image, cv2.COLOR_RGB2BGR)
        cv2.imwrite(os.path.join(overlays_dir, image_name), overlay_image)
        overlay_path = os.path.join(overlays_dir, image_name)

        # Add the overlay image to the PDF
        pdf.image(overlay_path, x=pdf.get_x(), y=pdf.get_y(), w=90)

        # Set the position for the original image
        pdf.set_xy(110, 8)

        image_rgb = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)
        cv2.imwrite(os.path.join(overlays_dir, "original_" + image_name), image_rgb)
        image_path = os.path.join(overlays_dir, "original_" + image_name)

        
        # Add the original image to the PDF
        pdf.image(image_path, x=pdf.get_x(), y=pdf.get_y(), w=90)


        #Set the position of the image of the mask
        pdf.set_xy(10, 110)
        image_of_masks = cv2.cvtColor(image_of_masks, cv2.COLOR_RGB2BGR)
        cv2.imwrite(os.path.join(overlays_dir, "image_of_masks_" + image_name), image_of_masks)
        image_of_masks_path = os.path.join(overlays_dir, "image_of_masks_" + image_name)

        # Add the image of the mask to the PDF
        pdf.image(image_of_masks_path, x=pdf.get_x(), y=pdf.get_y(), w=90)

        #read the image from the mask folder, it is the name of mask_folder + image_name + _segmentation.png
        mask = cv2.imread(mask_folder + image_name.split('.')[0] + '_segmentation.png')
        mask = cv2.resize(mask, (image.shape[1], image.shape[0]))
        # Add the mask to the PDF
        pdf.set_xy(110, 110)
        mask = cv2.cvtColor(mask, cv2.COLOR_RGB2BGR)
        cv2.imwrite(os.path.join(overlays_dir, "mask_" + image_name), mask)
        mask_path = os.path.join(overlays_dir, "mask_" + image_name)
        pdf.image(mask_path, x=pdf.get_x(), y=pdf.get_y(), w=90)


        

    # Save the PDF to a file
    pdf.output(os.path.join(overlays_dir, "processed_images_train.pdf"))





In [148]:
process_folder(real_images_folder, result_folder, mask_folder)

ValueError: could not broadcast input array from shape (396,396,3) into shape (271,271)

In [68]:
# mask_folder = './pad/train/mask/melanoma/'
# real_images_folder = './pad/train/data/melanoma/'
# result_folder = './result_sam/' 
# #get the first image from mask folder, then get the corresponding image from real_images_folder, then generate the mask and save the image with the mask on at result_folder
# image_name = os.listdir(real_images_folder)[0]


# image = cv2.imread(real_images_folder + image_name)
# #get the file name in mask folder that has image_name in it
# mask_name =  [mask_name for mask_name in os.listdir(mask_folder) if image_name[:-4] in mask_name][0]
# mask = cv2.imread(mask_folder + mask_name)
# #get the file name in result_folder that has image_name in it
# result_name = [result_name for result_name in os.listdir(result_folder) if image_name[:-4] in result_name][0]
# result = cv2.imread(result_folder + result_name)

# #plot the image, mask and result, plot the result with the mask on above it
# _,axes = plt.subplots(1,3, figsize=(16,16))
# axes[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
# axes[1].imshow(cv2.cvtColor(mask, cv2.COLOR_BGR2RGB))
# axes[2].imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
# show_output(mask, axes[2])
# plt.show()



In [ ]:


CHECKPOINT_PATH = "medsam_vit_b.pth"

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Device: {DEVICE}")
MODEL_TYPE = "vit_b"


sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH)
sam.to(device=DEVICE)
sam.eval()
mask_generator = SamAutomaticMaskGenerator(sam)




import os 

path = './pad/train/data/melanoma'
path_result = "result"
os.makedirs(path_result, exist_ok=True)

for filename in os.listdir(path):
    #if filename is at path_result, skip
    if os.path.exists(os.path.join(path_result, filename)):
        continue
    image_path = path + '/' + filename
    image = cv2.imread(image_path)
    #if image shape is greater than 1920x1080, resize 
    if image.shape[0] > 1080 or image.shape[1] > 1920:
        image = cv2.resize(image, (1920, 1080))

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    output_mask = mask_generator.generate(image)
    _,axes = plt.subplots(1,2, figsize=(16,16))
    axes[0].axis('off')
    axes[1].axis('off')
    axes[0].imshow(image_rgb)
    show_output(output_mask, axes[1])
    plt.savefig(os.path.join(path_result, filename))
    

